In [ ]:
#Imports
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#Comment this lines if you have this stuff already installed
#!(yes |pip install geopandas)
#!(yes |pip install descartes)
#!(yes |conda install -c conda-forge geoplot)
#!(yes | pip install plotly)
import plotly.express as px
#!pip install spacy
#!python -m spacy download en_core_web_sm
import nltk
from nltk.corpus import stopwords
from nltk.tag import pos_tag
#nltk.download('wordnet')
#nltk.download('punkt')
#nltk.download('stopwords')
import spacy
import en_core_web_sm
#!pip install gensim
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import KeyedVectors
from gensim import models
import gensim
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import re
#!pip install catboost
#!pip install xgboost
import xgboost as xgb
from catboost import CatBoostRegressor

%config IPCompleter.greedy=True
%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs
from sklearn.decomposition import PCA
from matplotlib import pyplot
import warnings
warnings.filterwarnings('ignore')
from numpy import linalg as LA
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.integrate import odeint
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn import tree
from gensim import models
from gensim.models import Word2Vec
!pip install pyenchant
import enchant
import string
import spacy
nlp = spacy.load("en_core_web_sm")


In [ ]:
#Gets the hashtag or user
def getter(text, char):
    pos = text.find(char)
    text = text[pos:]
    #Some users or hashtags finish with : or .
    if text.endswith(':') or text.endswith('.'):
        text = text[:-1]
    return text
def validUser(userName):
    if '@' in userName:
        user = getter(userName, '@')
        length = len(user)
        if (length > 1 and length <= 16):
            for char in user[1:]:
                if not(char.isalnum() or char == '_'): return False
            return True
    return False
def validLink(link):
    type1 = 'https://'
    type2 = 'http://'
    if type1 in link and len(link) > 9: return True
    if type2 in link and len(link) > 8: return True
    return False
def validHashtag(hashtag):
    if '#' in hashtag:
        hashtag = getter(hashtag, '#')
        hashtag = hashtag[1:]
        return hashtag.isalnum()
    return False
def analyzeTweets(text):
    result = [0,0,0] #number of usersTagged, hashtags and links
    text = text.split()
    for word in text:
        if validUser(word): result[0] += 1
        elif validHashtag(word): result[1] += 1
        elif validLink(word): result[2] += 1
    return result
def getInfo(dataList, pos):
    return dataList[pos]
def tweetSize(tweetLength, minValue, intervalRange):
    if tweetLength < (minValue + intervalRange): return 1 #Small
    if (minValue + intervalRange) <= tweetLength and tweetLength < (minValue + 2 * intervalRange): return 2 #Medium
    return 3 #Large

In [ ]:
world = pd.read_csv("./ToChangeKeywordsAndLocations/worldcities.csv", usecols = ['city_ascii','iso3' , 'country'])
worldIsoSet = set(world.iso3)
nlp = spacy.load("en_core_web_sm")
def findInDataset(locationText):
    location = re.sub(",|\.", "", locationText)
    for possibleCountry in location.split():
        if possibleCountry in  worldIsoSet: return 1
    return 0
def hasValidLocation(locationText):
    if pd.isnull(locationText) or locationText == 'unknown': return 0
    doc = nlp(locationText)
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            return 1
    return findInDataset(locationText)

In [ ]:
def formatDf(principalDf):
    df = principalDf.copy()
    df['aux_column'] = df.text.apply(analyzeTweets)
    df['users_tagged'] = df.aux_column.apply(getInfo,args=(0,))
    df['hashtags'] = df.aux_column.apply(getInfo,args=(1,))
    df['links'] = df.aux_column.apply(getInfo,args=(2,))
    del df['aux_column']
    df['tweet_length'] = df.text.str.len()
    minValue = df.tweet_length.min()
    maxValue = df.tweet_length.max()
    intervalRange = (maxValue - minValue) // 3
    df['tweet_size'] = df.tweet_length.apply(tweetSize, args = (minValue, intervalRange))
    df['Valid_location'] = df.location.apply(hasValidLocation)
    return df


In [ ]:
#Natural disasters
#Some auxiliar functions
def getSeriesElements(serie, setElements):
    for element in serie.values: #Element is a string always
        if '/' in element:
            element = element.split('/')
            for elemt in element: setElements.add(elemt.lower())
                
        else: setElements.add(element.lower())

#the info is between position 2 and 6, both included
def obtainInfo(infoList):
    naturalDisasters = {} #Key: group, value: {subgroups}
    for i in range (2,7): #To iterate the info in the list
        dataFrame = infoList[i]
        #Always delete the first row, it dosent have info
        dataFrame.drop(0, inplace = True)
        #The group always is at (0,1)
        group = dataFrame.iloc[0,1]
        #Now its time to iterate the columns of the DF
        cols = len(dataFrame.columns)
        subgroups = set()
        for col in range(2, cols):
            serie = dataFrame[col] #This is a serie
            serie.dropna(inplace=True)
            serie.drop_duplicates(inplace=True)
            getSeriesElements(serie, subgroups)
        naturalDisasters[group] = subgroups
    return naturalDisasters

def getDictWithDisasters():
    dataPage = pd.read_html('https://www.emdat.be/classification')
    naturalDisastersDicc = obtainInfo(dataPage)
    geo = naturalDisastersDicc['Geophysical']
    geo.update({'volcano', 'sinkhole', 'lava'})

    met = naturalDisastersDicc['Meteorological']
    met.update({'hurricane','typhoon','twister','cyclone','hailstorm',\
                'violent storm','rainstorm','sandstorm','snowstorm','windstorm'})
    met -= {'lightning','derecho','sand','wind'}

    hydro = naturalDisastersDicc['Hydrological']
    hydro.update({'debris','mudslide','avalanche','rockfall'})
    hydro.remove('avalanche (snow, debris, mudflow, rockfall)')

    clima = naturalDisastersDicc['Climatological']
    clima.update({'bush fire', 'land fire', 'brush fire'})
    clima.remove('land fire: brush, bush,  pasture')
    allNaturalDisasters = set()
    for value in naturalDisastersDicc.values():
        allNaturalDisasters = allNaturalDisasters.union(value)
    return allNaturalDisasters
def fixingKeywords(keyword):
    auxDictionary = {'floods':'flood', 'wild fires': 'wildfire', 'forest fires':'forest fire',\
                    'bush fires':'bush fire'}
    return auxDictionary.get(keyword, keyword)
def checkNaturalDisaster(text, hashToUse):
    for natDisas in hashToUse:
        if natDisas in text.lower(): return 1
    
    return 0

In [ ]:
def _getDisastersDf(df):
    allNaturalDisasters = getDictWithDisasters()
    disastersDf = df
    disastersDf.keyword = disastersDf.keyword.apply(fixingKeywords)
    natDisast1 = disastersDf.keyword.apply(lambda x: int(x in allNaturalDisasters)) #Basado en las keyword
    natDisast2 = disastersDf.text.apply(lambda x: checkNaturalDisaster(x, allNaturalDisasters)) #Basado en el texto
    disastersDf['Natural_disaster'] = natDisast1 | natDisast2
    return disastersDf
def getDisasterDf(df):
    disastersDf = _getDisastersDf(formatDf(df))
    disastersDf['Total_elements'] = disastersDf.hashtags + disastersDf.users_tagged + disastersDf.links
    disastersDf['links_hash'] = disastersDf.hashtags + disastersDf.links
    disastersDf['links_users'] = disastersDf.users_tagged + disastersDf.links
    disastersDf['hash_users'] = disastersDf.hashtags + disastersDf.users_tagged
    return disastersDf

In [ ]:
replacement_patterns = [
(r'won\'t', 'will not'),
(r'can\'t', 'cannot'),
(r'i\'m', 'i am'),
(r'I\'m', 'I am'),
(r'ain\'t', 'is not'),
(r'(\w+)\'ll', '\g<1> will'),
(r'(\w+)n\'t', '\g<1> not'),
(r'(\w+)\'ve', '\g<1> have'),
(r'(\w+)\'s', '\g<1> is'),
(r'(\w+)\'re', '\g<1> are'),
(r'(\w+)\'d', '\g<1> would')
]
class RegexpReplacer(object):
    def __init__(self, patterns=replacement_patterns):
        self.patterns = [(re.compile(regex), repl) for (regex, repl) in patterns]
    def replace(self, text):
        s = text
        for (pattern, repl) in self.patterns:
            (s, count) = re.subn(pattern, repl, s)
        return s
class RepeatReplacer(object):
    def __init__(self):
        self.repeat_regexp = re.compile(r'(\w*)(\w)\2(\w*)')
        self.repl = r'\1\2\3'
    def replace(self, word):
        if wordnet.synsets(word):
            return word
        repl_word = self.repeat_regexp.sub(self.repl, word)
        
        if repl_word != word:
            return self.replace(repl_word)

        return repl_word

In [ ]:
def cleanTweet(tweet, op = 1):
    if op:
        return ' '.join(re.sub("(\w+:\/\/\S+)", " ", tweet).split()) #Sino explota 
    
    return ' '.join(re.sub("((\w+:\/\/\S+)|\?|(#\w*))", " ", tweet).split())


def deletePunctuation(tokenizedText):
    x = re.compile('[%s]' % re.escape(string.punctuation))
    tokenized_text_no_punctuation = []
    for token in tokenizedText: #Agarro las palabras de la lista
        newToken = x.sub(u'', token)
        if not newToken == u'':
            tokenized_text_no_punctuation.append(newToken)
    return tokenized_text_no_punctuation
def deleteStopwords(tokenizedText, stopwords):
    return [word for word in tokenizedText if word not in stopwords]
def editText(text, stopwords, replacer, repeatReplacer):
    #Primero elimino los links
    text = cleanTweet(text)
    
    #Paso a lower el text
    text = text.lower()
    
    #Reemplazo los I'm por I am
    text = replacer.replace(text)
    
    #Elimino los caracteres repetidos, ej: ohhh por oh
    words = text.split()
    text = ' '.join(repeatReplacer.replace(word) for word in words)
    
    #Tokenizo el texto
    tokenizedText = nltk.word_tokenize(text)
    
    #Elimno los signos de puntuacion
    tokenizedText = deletePunctuation(tokenizedText)
    
    #Elimino los stopwords
    tokenizedText = deleteStopwords(tokenizedText, stopwords)
    
    editText = ' '.join(tokenizedText)
    return editText

englishDict = enchant.Dict('en_US')

def spellChecker(text):
    error = 0
    amountOfWords = 0
    text = cleanTweet(text, 0)
    sentenceTokenized = nltk.word_tokenize(text)
    for word in sentenceTokenized:
        if word.isalnum():
            amountOfWords += 1
            if not englishDict.check(word): error += 1
    
    return np.round(100 * error / amountOfWords) if amountOfWords else amountOfWords
def getOp(value, operation, vectorOp = False):
    if vectorOp:
        finalValue = 0
        for word in value.split():
            if word in model.vocab:
                finalValue = operation(model.get_vector(word), finalValue)
        return finalValue
        
    finalValue = [operation(model.get_vector(i)) for i in value.split() if i in model.vocab]
    return finalValue if len(finalValue) and not np.isnan(finalValue).any() else 0

def languageAnalysis(row):
    words = nlp(row.text)
    stopTrue = 0
    alphaTrue = 0
    verbs = 0
    adverbs = 0
    nouns = 0
    pronouns = 0
    for token in words:
        stopTrue += 1 if token.is_stop else 0
        alphaTrue += 1 if token.is_alpha else 0
        verbs += 1 if token.tag_.startswith('VB') else 0
        pronouns += 1 if token.tag_.startswith('PR') else 0
        nouns += 1 if token.tag_.startswith('NN') else 0
        adverbs += 1 if token.tag_.startswith('RB') else 0
    row['verbs'], row['stopTrue'], row['alphaTrue'] = verbs, stopTrue, alphaTrue
    row['pronouns'], row['nouns'], row['adverbs'] = pronouns, nouns, adverbs
    return row


In [ ]:
dataPage = pd.read_html('https://es.wikipedia.org/wiki/Anexo:Cuentas_de_Twitter_con_m%C3%A1s_seguidores')
top100Users = {username.lower()[1:] for username in dataPage[0].iloc[:100,1]}
def hasTopUser(text):
    words = text.split()
    for word in words:
        if word in top100Users: return 1
    return 0

In [ ]:
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
file = 'GoogleNews-vectors-negative300.bin.gz'

model = KeyedVectors.load_word2vec_format (file, binary=True)
def getSumWithText(disastersDf):
    disastersDf['textProm'] = disastersDf.text.apply(lambda x: np.mean(getOp(x, sum)))
    disastersDf['textMeanStd'] = disastersDf.text.apply(lambda x: np.mean(getOp(x, np.std)))
    disastersDf['textStd'] = disastersDf.text.apply(lambda x: np.std(getOp(x, np.add, True)))
    disastersDf['textMean'] = disastersDf.text.apply(lambda x: np.mean(getOp(x, np.add, True)))
    disastersDf['textMeanMean'] = disastersDf.text.apply(lambda x: np.mean(getOp(x, np.mean)))

def modifyDfText(df):
    dfToReturn = df.copy()
    dfToReturn['error_percentage'] = dfToReturn.text.apply(lambda x: spellChecker(x))
    dfToReturn['Porn_words'] = dfToReturn.text.apply(pornWords)
    repeatReplacer = RepeatReplacer()
    replacer = RegexpReplacer()
    stop = set(stopwords.words('english'))
    dfToReturn['text'] = dfToReturn.text.apply(editText, args = (stop, replacer, repeatReplacer))
    return dfToReturn

def operateWithText(df):
    '''This method will return a copy of df with the text changed and cols with the prom std and mean'''
    dfToReturn  = modifyDfText(df)
    dfToReturn['hasTopUser'] = dfToReturn.text.apply(hasTopUser)
    getSumWithText(dfToReturn)
    return dfToReturn.apply(languageAnalysis, axis = 1, result_type = 'expand')
    

In [ ]:
def standarNorm(dataFrame, labels):
    normMethod = '_standardNorm'
    for label in labels:
        mean = dataFrame[label].mean()
        std = dataFrame[label].std()
        dataFrame[label + normMethod] = dataFrame[label].apply(lambda x: (x - mean) / std)

def minMaxNorm(dataFrame, labels):
    normMethod = '_minMaxNorm'
    for label in labels:
        minValue = dataFrame[label].min()
        maxValue = dataFrame[label].max()
        dataFrame[label + normMethod] = dataFrame[label].apply(lambda x: (x - minValue) / (maxValue - minValue))

def tweetElementsNorm(dataFrame, labels):
    labels = ['hashtags', 'users_tagged', 'links', 'Porn_words',\
             'links_hash', 'links_users', 'hash_users']
    normMethod = '_extraNorm'
    dataFrame['auxCol'] = dataFrame.text.apply(lambda x: len(x.split()) if len(x.split()) else 1)
    for label in labels:
        if label == 'links': #Caso borde porque volamos los links del text
            dataFrame[label + normMethod] = dataFrame[label] / (dataFrame.auxCol + dataFrame.links)
            continue            
            
        dataFrame[label + normMethod] = dataFrame[label] / dataFrame.auxCol
    del dataFrame['auxCol']


def applyNormalizations(dataFrame):
    col = 5 if 'target' in set(dataFrame.columns) else 4 #Para evitar problemas con el test.csv
    substract = {'textProm', 'textMeanStd', 'textStd', 'textMean', 'norm2', 'Valid_location', 'Natural_disaster'}
    normalization = [standarNorm, minMaxNorm, tweetElementsNorm]
    labels = list(set(dataFrame.iloc[1:2, col:].columns) - substract)
    for normOp in normalization:
        normOp(dataFrame, labels)
def norm2(text):
    norm = 0
    for word in text.split():
        if word in model.vocab:
            norm += np.linalg.norm(model.get_vector(word))
    return norm
def getVectors(text):
    value = np.array([float(0) for i in range(300)])
    for word in text.split():
        if word in model.vocab:
            value += model.get_vector(word)
    return value

def _getAppearancesHash(df):
    appearances = {}
    for word in df.keyword.to_list():
        appearances[word] = appearances.get(word, 0) + 1 if not pd.isnull(word) else 0
    return appearances

def _getAppearencesKeyword(row, hashToUse):
    row['keywordAppearance'] = hashToUse[row['keyword']]
    return row

def getAppearenceInDf(df):
    appearences = _getAppearancesHash(df)
    return df.apply(lambda x: _getAppearencesKeyword(x, appearences), axis = 1, result_type = 'expand')
    

In [ ]:
pornwords = {'sex','sexy', 'cunt', 'dick', 'cock', 'xxx', 'porn',\
                 'lesbian', 'gay', 'masturbation', 'fap', 'asshole',\
                 'assholes', 'suck', 'sucker', 'idiot', 'stupid', 'cum',\
                 'blowjob', 'bitch', 'slut', 'sluts', 'whores', 'bitches', 'whore',\
                 'cunts', 'suckers', 'ass', 'butt', 'nude', 'nudes', 'naked', 'fucking',\
                 'xoxo', 'cocks', 'dicks', 'wtf', 'lol', 'lmfao', 'lmao', 'cunts', 'jerkface'}

def pornWords(sentence):
    value = 0
    for word in sentence.split():
        if word.lower() in pornwords:
            value += 1
    
    return value

In [ ]:
import os.path
from os import path
def main(df):
    disastersDf = operateWithText(getDisasterDf(df))
    disastersDf['length_proportion'] = disastersDf.text.str.len() / disastersDf.tweet_length
    disastersDf['amount_of_words_proportion'] = disastersDf.text.apply(lambda x: len(x.split())) / df.text.apply(lambda x: len(x.split()))
    applyNormalizations(disastersDf)
    disastersDf['norm2'] = disastersDf.text.apply(norm2)
    disastersDf = disastersDf.join(disastersDf.apply(lambda x: getVectors(x['text']), axis = 1, result_type = 'expand'))
    disastersDf = getAppearenceInDf(disastersDf)
    disastersDf['isRelevant'] = getRelevantTweetsDf(disastersDf)['isRelevant']
    return disastersDf

In [ ]:
import requests
from urllib.parse import quote
textForRequest = "https://api.dbpedia-spotlight.org/en/annotate?text="
def getRelevantTweets(row):
    text = row['text']
    response = requests.get(textForRequest + quote(text), verify=False)
    row['isRelevant'] = bool(re.findall('href=".*title', response.text))
    return row
#isRelevantCsv = pd.read_csv('train.csv').apply(getRelevantTweets, axis = 1, result_type='expand')
#isRelevantCsv.to_csv('trainWithIsRelevant.csv', index = False)
#These lines are for the solely purpose if anybody want to have this method run for over an hour for something that we should already have

In [ ]:
def getRelevantTweetsDf(df): 
    return pd.read_csv('trainWithIsRelevant.csv') if path.exists('trainWithIsRelevant.csv') else df.apply(getRelevantTweets, axis = 1, result_type='expand')
